# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770867182381                   -0.53    9.0         
  2   -2.772146256505       -2.89       -1.32    1.0    172ms
  3   -2.772170476189       -4.62       -2.52    1.0    213ms
  4   -2.772170684073       -6.68       -3.35    1.0    179ms
  5   -2.772170721687       -7.42       -3.95    2.0    214ms
  6   -2.772170722971       -8.89       -5.01    1.0    210ms
  7   -2.772170723014      -10.37       -5.46    2.0    210ms
  8   -2.772170723015      -11.99       -6.33    2.0    215ms
  9   -2.772170723015      -14.31       -6.68    1.0    221ms
 10   -2.772170723015      -13.88       -7.58    1.0    216ms
 11   -2.772170723015   +  -14.12       -8.55    2.0    257ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770762626025                   -0.52    9.0         
  2   -2.7

1.7735581673163774

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770626759424                   -0.53    8.0         
  2   -2.772047136407       -2.85       -1.30    1.0    227ms
  3   -2.772082405733       -4.45       -2.65    1.0    183ms
  4   -2.772083413486       -6.00       -3.78    2.0    240ms
  5   -2.772083417173       -8.43       -4.17    2.0    209ms
  6   -2.772083417796       -9.21       -5.50    1.0    203ms
  7   -2.772083417810      -10.84       -5.90    2.0    217ms
  8   -2.772083417811      -12.89       -6.42    1.0    218ms
  9   -2.772083417811      -13.51       -7.14    2.0    247ms
 10   -2.772083417811   +  -14.35       -7.39    1.0    241ms
 11   -2.772083417811   +  -14.24       -8.43    1.0    199ms

Polarizability via ForwardDiff:       1.7725349723395332
Polarizability via finite difference: 1.7735581673163774
